# Goal of script: 
Very simple SIR-modelling to Omikron wave, to get a very rough estimate of wave end

In [1]:
%matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns
from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


# Get data 

In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [3]:
latestsubdir = list(os.walk(path_data))[0][1][-1]
latestdir = path_data + latestsubdir

dfCase = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
dfCase = dfCase.iloc[:-2]
dfCase['NewPositive'] = pd.to_numeric(dfCase['NewPositive'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['Tested'] = pd.to_numeric(dfCase['Tested'].astype(str).apply(lambda x: x.replace('.','')))
dfCase['PosPct'] = pd.to_numeric(dfCase['PosPct'].astype(str).apply(lambda x: x.replace(',','.')))
dfCase['Date'] =  pd.to_datetime(dfCase.Date,format='%Y-%m-%d')
testDates = dfCase['Date']

dfAdm = pd.read_csv(latestdir+'/Newly_admitted_over_time.csv',delimiter = ';',dtype=str)
dfAdm['Dato'] = pd.to_datetime(dfAdm['Dato'])
dfAdm['Total'] = pd.to_numeric(dfAdm['Total'])
dfAdm.tail()


dfDea = pd.read_csv(latestdir+'/Deaths_over_time.csv',delimiter = ';',dtype=str)
dfDea = dfDea.iloc[:-1,:]
dfDea['Dato'] = pd.to_datetime(dfDea['Dato'])
dfDea['Antal_døde'] = pd.to_numeric(dfDea['Antal_døde'])
dfDea.tail()


,Dato,Antal_døde
695,2022-02-04,20
696,2022-02-05,18
697,2022-02-06,27
698,2022-02-07,8
699,2022-02-08,0


In [4]:
allDates = dfCase.Date
allDatesAdm = dfAdm.Dato
allDatesDea = dfDea.Dato

allCases = dfCase.NewPositive.values

firstDate = np.datetime64('2021-10-01')-np.timedelta64(1,'D')
# lastDate = np.datetime64('2022-03-01')+np.timedelta64(1,'D')
lastDate = np.datetime64('2022-03-01')

meanWidth = 7

In [5]:
# fig,ax1 = plt.subplots(tight_layout=True)

# ax1.plot(allDates[:-1],allCases[:-1],'b.:',markersize=6,linewidth=0.75,label='Daglig data')
# ax1.plot(rnTime(allDates[:-1],meanWidth),rnMean(allCases[:-1],meanWidth),'b',linewidth=5,label=f'{meanWidth} dages gennemsnit')

In [6]:
# SIR-model, with just beta and gamma
def SIRmodel(t,x,beta,gamma):

    S,I = x

    dS = - beta * S * I 
    dI =   beta * S * I - gamma * I

    return [dS,dI]

In [7]:
# Model test
R0 = 0.2
I0 = 0.001
S0 = 1 - R0 - I0 

beta0 = 0.4
gamma0 = 1/7
pars0 = (beta0,gamma0)

r_naught_0 = beta0/gamma0


t0 = 0
tEnd = 100
tRange = np.arange(t0,tEnd)

# sol = solve_ivp(SIRmodel,[t0,tEnd],[S0,I0],args=pars0,dense_output=True)
sol = solve_ivp(SIRmodel,[t0,tEnd],[S0,I0],t_eval=tRange,args=pars0)

curT = sol.t
curS = sol.y[0,:]
curI = sol.y[1,:]
# sol.y[0,:]
plt.figure(figsize=(10,5))

plt.plot(curT,curS)
plt.plot(curT,curI)
plt.plot(curT,1-curS-curI)

plt.ylim(bottom=0,top=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 1.0)

In [8]:
# Convert data to something comparable
# Use running means
meanWidth = 7
rnCases = rnMean(allCases,meanWidth)
rnDates = rnTime(allDates.values,meanWidth)

avgInfPeriod = 7
dataR = rnCases.cumsum()[avgInfPeriod-1:]
dataI = np.convolve(rnCases, np.ones(avgInfPeriod), mode='valid')
dataDates = rnDates[avgInfPeriod-1:]

# Assume only a fraction of cases have been detected
detect_rate = 0.85
dataR = dataR/detect_rate
dataI = dataI/detect_rate

dkpop = 5831000

dataS = dkpop-dataR-dataI

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
ax1.plot(dataDates,dataS,'b')
ax1.plot(dataDates,dataI,'r')
ax1.plot(dataDates,dataR,'g')

ax2.plot(dataDates,dataI,'r')
ax3.plot(dataDates,dataR,'g')

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=np.datetime64('2021-10-01'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(18901.0, 19063.5)

In [9]:
# # dataDates[modelStartIndex]
# np.where(dataDates == modelStartDate) 
# modelStartIndex[0]

In [10]:
# Small test
modelStartDate = np.datetime64('2021-12-01')
modelStartDate = np.datetime64('2022-01-12')
# modelStartIndex = dataDates[dataDates == modelStartDate].index[0]
modelStartIndex = np.where(dataDates == modelStartDate)[0][0]

R_init = dataR[modelStartIndex]/dkpop
I_init = dataI[modelStartIndex]/dkpop
S_init = dataS[modelStartIndex]/dkpop

# Make time array
t0 = 0
tEnd = 100
tRange = np.arange(t0,tEnd)
dRange = modelStartDate + np.arange(np.timedelta64(t0,'D'),np.timedelta64(tEnd,'D'))
# dRange = dRange + np.timedelta64(int(np.floor(avgInfPeriod/2))-1,'D')

# Parameters
# r_naught_0 = 4.7 # Arbritarily chosen
# r_naught_0 = 1.9 # Arbritarily chosen
r_naught_0 = 1.5 # Arbritarily chosen
r_naught_0 = 2 # Arbritarily chosen
r_naught_0 = 2.05 # Arbritarily chosen
gamma0 = 1/avgInfPeriod
beta0 = r_naught_0*gamma0
pars0 = (beta0,gamma0)

# sol = solve_ivp(SIRmodel,[t0,tEnd],[S0,I0],args=pars0,dense_output=True)
sol = solve_ivp(SIRmodel,[t0,tEnd],[S_init,I_init],t_eval=tRange,args=pars0)

curS = sol.y[0,:]*dkpop
curI = sol.y[1,:]*dkpop
curR = dkpop - curS - curI

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,sharex=True,figsize=(12,20))
ax1.plot(dataDates,dataS,'b')
ax1.plot(dataDates,dataI,'r')
ax1.plot(dataDates,dataR,'g')
ax1.plot(dRange,curS,'k--')
ax1.plot(dRange,curR,'k--')
ax1.plot(dRange,curI,'k--')

ax2.plot(dRange,curR,'k')
ax2.plot(dataDates,dataR,'g.-',markersize=2,linewidth=1)

ax3.plot(dRange,curI,'k')
ax3.plot(dataDates,dataI,'r.-',markersize=2,linewidth=1)



ax4.plot(allDates[:-1],allCases[:-1],'.:',color='xkcd:dark red',linewidth=0.5,markersize=2)
ax4.plot(rnTime(allDates[:-1],7),rnMean(allCases[:-1],7),color='xkcd:dark red')

convPer = avgInfPeriod + 1
convPer = avgInfPeriod 
halfConvPer = int(np.floor(convPer/2))
deconv,_ = signal.deconvolve(curI,np.ones(convPer))
# ax4.plot(rnTime(dRange[:-(convPer-1)],convPer),rnMean(deconv,convPer),'k:')
# ax4.plot(rnTime(dRange[halfConvPer+1:-halfConvPer+1],convPer),rnMean(deconv,convPer),'k')
ax4.plot(rnTime(dRange[halfConvPer:-halfConvPer],convPer),rnMean(deconv,convPer),'k')

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0)
ax4.set_ylim(bottom=0)

ax2.set_ylim(top = np.max(dataR)*1.5)
# ax3.set_ylim(top = np.max(dataI)*1.1)

# Draw weekends
firstSunday = np.datetime64('2021-10-03')
numWeeks = 52
for k in range(-numWeeks,numWeeks):
     curSunday = firstSunday + np.timedelta64(7*k,'D')
     ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
     ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
     ax3.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
     ax4.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='lightgrey',label=int(k==0)*'Weekend')
ax1.grid(axis='y')


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=np.datetime64('2021-11-01'))
ax1.set_xlim(right=np.datetime64('2022-04-01'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'signal' is not defined

In [11]:
# # Comparison of daily infections
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

# ax1.plot(dataDates,dataI)
# ax1.plot(dRange,curI,':')
# ax2.plot(allDates[:-1],allCases[:-1])
# ax2.plot(rnTime(allDates[:-1],7),rnMean(allCases[:-1],7))

# convPer = avgInfPeriod + 1
# deconv,_ = signal.deconvolve(curI,np.ones(convPer))
# ax2.plot(rnTime(dRange[:-(convPer-1)],convPer),rnMean(deconv,convPer))
# ax2.set_xlim(left=np.datetime64('2021-10-01'))

In [12]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(dataDates,dataI)
ax1.plot(dRange,curI,':')
ax2.plot(allDates[:-1],allCases[:-1])
ax2.plot(rnTime(allDates[:-1],7),rnMean(allCases[:-1],7))

# deconv,_ = signal.deconvolve(dataI,np.ones(avgInfPeriod))
# ax2.plot(dataDates[:-(avgInfPeriod-1)],deconv)

# ax2.plot(dataDates[:-avgInfPeriod],dataI[avgInfPeriod:]-dataI[:-avgInfPeriod])
# ax2.plot(dataDates[:-1],dataI[1:]-dataI[:-1])
convPer = avgInfPeriod + 1
deconv,_ = signal.deconvolve(curI,np.ones(convPer))
ax2.plot(rnTime(dRange[:-(convPer-1)],convPer),rnMean(deconv,convPer))
# ax2.plot(dRange[:-(avgInfPeriod-1)],deconv)

ax2.set_xlim(left=np.datetime64('2021-10-01'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'signal' is not defined

In [13]:
fig,(ax1,ax2) = plt.subplots(2,1)

ax1.plot(dataDates,dataI)

ax2.plot(allDates,allCases)

from scipy import signal 

# dataI = np.convolve(rnCases, np.ones(avgInfPeriod), mode='valid')

deconv,_ = signal.deconvolve(dataI,np.ones(avgInfPeriod+1))
ax2.plot(dataDates[:-(avgInfPeriod)],deconv)
ax2.plot(rnTime(allDates,7),rnMean(allCases,7),'k')

ax2.set_xlim(left=np.datetime64('2021-10-01'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(18901.0, 19067.1)

In [14]:
# Try something realistic
modelStartDate = np.datetime64('2022-01-12')
modelStartDate = np.datetime64('2021-12-05')
# modelStartDate = np.datetime64('2022-01-05')
modelStartDate = np.datetime64('2022-01-10')
modelStartIndex = allDates[allDates == modelStartDate].index[0]

caseSum = allCases.cumsum()

# Assume only a fraction of cases have been detected
detect_rate = 0.85
# detect_rate = 0.65

R_init = caseSum[modelStartIndex]/detect_rate

avgInfPeriod = 5
# avgInfPeriod = 7
I_init = allCases[modelStartIndex-avgInfPeriod:modelStartIndex].sum()
I_data = np.convolve(allCases, np.ones(avgInfPeriod), mode='valid')

# I_init = I_data[modelStartIndex-int(np.floor(avgInfPeriod/2))]
# R_init = caseSum[modelStartIndex-int(np.floor(avgInfPeriod/2))]/detect_rate

# Scale by population
dkpop = 5831000
R_init = R_init/dkpop
I_init = I_init/dkpop

S_init = 1 - R_init - I_init

# r_naught_0 = 4.7 # Arbritarily chosen
r_naught_0 = 2.5 # Arbritarily chosen
r_naught_0 = 1.9 # Arbritarily chosen
# r_naught_0 = 1.5 # Arbritarily chosen
gamma0 = 1/avgInfPeriod
beta0 = r_naught_0*gamma0
pars0 = (beta0,gamma0)

# Make simulation
t0 = 0
tEnd = 100
tRange = np.arange(t0,tEnd)
dRange = modelStartDate + np.arange(np.timedelta64(t0,'D'),np.timedelta64(tEnd,'D'))
dRange = dRange + np.timedelta64(int(np.floor(avgInfPeriod/2))-1,'D')

# sol = solve_ivp(SIRmodel,[t0,tEnd],[S0,I0],args=pars0,dense_output=True)
sol = solve_ivp(SIRmodel,[t0,tEnd],[S_init,I_init],t_eval=tRange,args=pars0)

curT = sol.t
curS = sol.y[0,:]
curI = sol.y[1,:]

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

# ax1.plot(dRange,curS)
# ax1.plot(dRange,curI)
ax1.plot(allDates,(caseSum/dkpop)/detect_rate,'k.:',linewidth=0.5,markersize=3)
ax1.plot(dRange,1-curS-curI)
# ax1.plot(allDates[(avgInfPeriod-1):],(I_data/dkpop),'.')
# ax1.plot(modelStartDate,I_init,'*')
# ax1.set_ylim(bottom=0,top=1)
ax1.set_ylim(bottom=0)

ax2.plot(allDates[(avgInfPeriod-1):],(I_data/dkpop),'k.:',linewidth=0.5,markersize=3)
ax2.plot(rnTime(allDates[(avgInfPeriod-1):],meanWidth),rnMean((I_data/dkpop),meanWidth),'k--')
ax2.plot(dRange,curI)

# ax1.plot(dRange,dkpop*curS)
# ax1.plot(dRange,dkpop*curI)
# ax1.plot(dRange,dkpop*(1-curS-curI))
# ax1.plot(allDates,(caseSum)/detect_rate,'.')
# ax1.plot(allDates[(avgInfPeriod-1):],(I_data),'.')
# ax1.plot(modelStartDate,I_init,'*')
# ax1.set_ylim(bottom=0,top=dkpop)


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=modelStartDate-np.timedelta64(30,'D'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(18972.0, 19142.7)

In [11]:
int(np.floor(avgInfPeriod/2))


3